## Import Google Drive

In [14]:
# We'll first need the data from my drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Packages

In [2]:
# Need both chatterbot and chatterbot-corpus
!pip install chatterbot -qq
!pip install chatterbot-corpus -qq
# The above libraries compatible with earlier SpaCy. Need to download from what they ^ install.
!python -m spacy download en_core_web_sm -qq
# Using env variables, need this library
!pip install colab-env --upgrade
# For the API
!pip install amadeus

     |████████████████████████████████| 71kB 2.0MB/s 
     |████████████████████████████████| 30.9MB 125kB/s 
     |████████████████████████████████| 204kB 47.8MB/s 
     |████████████████████████████████| 5.7MB 43.9MB/s 
     |████████████████████████████████| 235kB 47.5MB/s 
     |████████████████████████████████| 266kB 48.8MB/s 
     |████████████████████████████████| 92kB 10.9MB/s 
     |████████████████████████████████| 2.1MB 42.5MB/s 
     |████████████████████████████████| 3.2MB 45.5MB/s 
ERROR: fbprophet 0.6 has requirement python-dateutil>=2.8.0, but you'll have python-dateutil 2.7.5 which is incompatible.
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.9 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 122kB 2.8MB/s 
     |████████████████████████████████| 276kB 8.4MB/s 
ERROR: chatterbot 1.0.5 has requirement

In [1]:
path='data/'
#import colab_env
import os
import datetime
from dotenv import load_dotenv
load_dotenv(dotenv_path='')
# The API
from amadeus import Client, ResponseError

# This library can extract dates from text
import dateutil.parser as dparser

# We'll need the following nltk packages
import nltk
nltk.download('punkt')
nltk.download('wordnet')

# We'll use lemmas instead of stems.
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# We will tag the parts of the speech 
from nltk.tag import pos_tag

# Get the stop words
from nltk.corpus import stopwords
nltk.download('stopwords')

# We'll need json to open a file of intents
import json

# We will need to save the model
import pickle

# Some helper libraries
import numpy as np
import pandas as pd
import random
import re

# We'll need these to actually create a machine learning model.
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD, Adagrad, Adam, Adadelta

# We may need to try out a grid search
from sklearn.model_selection import GridSearchCV

# We'll need this for the in-code tagging of words
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.matcher import Matcher

from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
from chatterbot.trainers import ChatterBotCorpusTrainer

[nltk_data] Downloading package punkt to /home/karl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/karl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/karl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Greetings and Farewells

In [2]:
# Greetings
hellos = [['Hello!','Hi there'],
          ['Hi there','Hello!'],
          ['Greetings','Hey'],
          ['Hey','Oh hey!'],
          ['Good morning','Good morning'],
          ['Good evening','Good evening'],
          ['Hey there!','Well hello there']
         ]

# Farewells
byes = [['Bye','Goodbye!'],
        ['See ya','Good bye!'],
        ['Have a good night','You as well, goodbye!'],
        ['Farewell','Later!'],
        ['Goodbye!', 'Bye for now!']
       ]

## Basic Conversations

In [3]:
convo1=["I would like to book a flight.",
        "I can help you with that. Where are you traveling to?",
        "I am traveling to Singapore.",
        "What date will you be traveling?",
        "I want to fly on June 14th.",
        "When would you like to return?"]

convo2=['I want to buy a plane ticket.',
        'I can help you make your reservation. What is your destination?',
        'My final destination is Sydney, Australia.',
        'What is your travel date?',
        'I am making a reservation for December 12th.',
        'Fantastic, when will you need to come back?']

convo3=['I need to make a plane reservation.',
        'We can book your trip right now. What city are you flying to?',
        'I need to fly to New York City.',
        'What date would you like me to book this plane ticket(s) for?',
        'I need a flight on July 4th.',
        "Awesome, when do you think you'd like to return?"]

convo4=['I want to take a vacation',
        'We can help you with that! What were you thinking?',
        'I was thinking Dominica.',
        'When would you like to leave?',
        'I need a flight on March 3rd.',
        "Let's get booking then! I'll need your return date."]
convo5=['I need a goddamn vacation',
        'Where do you come from, where do you go? Where are you coming from, Cotton Eyed Joe?',
        "That was random, but I think I'll check out London",
        "Radical, let's get to booking! When did you want to leave?",
        "Probably this week, June 29th, 2020",
        "Excellent, I only have a few more questions for you."
        ]

convos = [convo1,convo2,convo3, convo4, convo5]

## Training the Chatbot

For this, we'll actually also import the english corpus for a little more flexibility in what the chatbot can say. That way it can hold a brief conversation before booking the flight.

In [4]:
# We will first train the bot with the lists of data we've given it above.
chatbot = ChatBot('Traveller',logic_adapters=["chatterbot.logic.BestMatch"])
travel_listtrainer = ListTrainer(chatbot)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/karl/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/karl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/karl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
for convo in convos:
  travel_listtrainer.train(convo)

for hi in hellos:
  travel_listtrainer.train(hi)

for bye in byes:
  travel_listtrainer.train(bye)

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%


In [6]:
# Next we will need to train in with the corpus.
travel_corpustrainer = ChatterBotCorpusTrainer(chatbot)

In [7]:
travel_corpustrainer.train("chatterbot.corpus.english")

Training ai.yml: [                    ] 1%

/home/karl/Documents/chatbot/ChatBot_1000ml_p7/cb_env/lib/python3.8/site-packages/chatterbot/corpus.py:38: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(data_file)


Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


## Attempt to make functions more streamlined

What I want to do here is to make it so that everytime a user inputs a message, the chatbot looks for the entities that it needs. So we will start with the looker function

### Looker Function

In [27]:
def entity_looker(message):
    '''
    This function looks at a message from the user and seeks for matches in dates, locations and budget
    and outputs a value of True if they are contained in the message. 
    '''
    
    # Tokenize the sentence for entity matching.
    tokens = nlp(message)
    
    date_from_matcher = Matcher(nlp.vocab)
    date_from_matcher.add("DateFromFinder", None, [{'LOWER':'leave'}, {'LOWER':'on'}, {"ENT_TYPE": "DATE"}],
                                                  [{'LOWER':'leaving'}, {'LOWER':'on'}, {"ENT_TYPE": "DATE"}],
                                                  [{'LOWER':'leave'}, {"ENT_TYPE": "DATE"}],
                                                  [{'LOWER':'leaving'}, {"ENT_TYPE": "DATE"}],
                                                  [{'LOWER':'go'}, {'LOWER':'on'}, {"ENT_TYPE": "DATE"}],
                                                  [{'LOWER':'on'}, {"ENT_TYPE": "DATE"}], 
                                                  [{'LOWER':'from'}, {"ENT_TYPE": "DATE"}])
    
    date_to_matcher = Matcher(nlp.vocab)
    date_to_matcher.add("DateToFinder", None, [{'LOWER':'return'},{'LOWER':'on'}, {"ENT_TYPE": "DATE"}],
                                              [{'LOWER':'returning'},{'LOWER':'on'}, {"ENT_TYPE": "DATE"}],
                                              [{'LOWER':'return'},{"ENT_TYPE": "DATE"}],
                                              [{'LOWER':'returning'}, {"ENT_TYPE": "DATE"}],
                                              [{'LOWER':'back'},{'LOWER':'on'}, {"ENT_TYPE": "DATE"}],
                                              [{'LOWER':'to'}, {"ENT_TYPE": "DATE"}])
    
    origin_matcher = Matcher(nlp.vocab)
    origin_matcher.add("OriginFinder", None, [{'LOWER':'from'},{'ENT_TYPE':'GPE'}],
                                             [{'LOWER':'of'},{'ENT_TYPE':'GPE'}],
                                             [{'LOWER':'leaving'},{'ENT_TYPE':'GPE'}],
                                             [{'LOWER':'leave'},{'ENT_TYPE':'GPE'}])
    dest_matcher = Matcher(nlp.vocab)
    dest_matcher.add("DestinationFinder", None, [{'LOWER':'to'},{'ENT_TYPE':'GPE'}],
                                                [{'LOWER':'visit'},{'ENT_TYPE':'GPE'}],
                                                [{'LOWER':'see'},{'ENT_TYPE':'GPE'}],
                                                [{'LOWER':'out'},{'ENT_TYPE':'GPE'}],
                                                [{'LOWER':'travel'},{'ENT_TYPE':'GPE'}])
    
    budget_matcher = Matcher(nlp.vocab)
    budget_matcher.add('BudgetFinder', None, [{'LOWER':'of'},{'IS_DIGIT':True}],
                                             [{'LOWER':'spend'},{'IS_DIGIT':True}],
                                             [{'LOWER':'under'},{'IS_DIGIT':True}],
                                             [{'LOWER':'than'},{'IS_DIGIT':True}],
                                             [{'ENT_TYPE':'MONEY'}],
                                             [{'ORTH':'$'},{'IS_DIGIT':True}])
    
    # This portion uses the boolean nature of an empty list. If the list is empty, we get False, else True
    # Date From Match
    DF_match = not not date_from_matcher(tokens)
    
    # Date To Match
    DT_match = not not date_to_matcher(tokens)
    
    # Origin Match
    O_match = not not origin_matcher(tokens)
    
    # Destination Match
    D_match = not not dest_matcher(tokens)
    
    # Budget Match
    B_match = not not budget_matcher(tokens)
    
    # This returns a tuple of the Trues/Falses, so the functions know what they are looking for specifically.
    return (DF_match, DT_match, O_match, D_match, B_match)

### Picker Function

In [38]:
def entity_picker(message):
    '''
    This function takes the match information, and calls on the functions necessary to extract the info
    from the message. It then converts the entities that it picked out to an API readable form.
    It then returns a set of the entities. If there were no matches, it returns None.
    '''
    
    matches = entity_looker(message)
    from_date = to_date = origin_code = dest_code = budget_int = None
    
    # From Date match
    if matches[0]:
        # Get the from date (comes back as DateTime)
        from_date_dt = get_from_dates(message)
        # Convert dates for API
        from_date = convert_date(from_date_dt)
    
    # To Date match
    if matches[1]:
        # Get the from date (comes back as DateTime)
        to_date_dt = get_to_dates(message)
        # Convert dates for API
        to_date = convert_date(to_date_dt)
        
    # Origin Location match
    if matches[2]:
        # Get the origin location
        origin = get_origin(message)
        # convert to IATA code
        origin_code = iata_code(origin)
    
    # Destination Location match
    if matches[3]:
        # get the destination location
        dest = get_dest(message)
        # convert to IATA code
        dest_code = iata_code(dest)
    
    # Budget match
    if matches[4]:
        # get the budget
        budget = get_budget(message)
        # convert to int
        budget_int = int(budget)
    
    return (from_date, to_date, origin_code, dest_code, budget_int)

### From Date Extractor

In [51]:
def get_from_dates(message):
    '''
    This function is designed to extract a datetime object from the "from date". Here it is also necessary to 
    distinguish between the from/to dates if they are both included in a single message. Thankfully, SpaCy lists
    them in order of appearance. 
    It will be an important note that for the purposes of this project, I will _assume_ that (if there are two
    dates present) the date which  appears first is the from date, and the second is the to date. There is no 
    good reason that proper English would lead someone to list both dates in the reverse order.
    This function assumes that there is at least one date present in the message.
    '''
    import datetime
    # Tokenize
    tokens = nlp(message)
    # Extract entities
    entities = [(X.label_, X.text) for X in tokens.ents]
    # get rid of anything not a date
    dates = [x for x in entities if x[0]=='DATE']
    
    # Next, we will look to see if someone used the words "tomorrow", "next week" or "today"
    text = [x[1] for x in dates]
    today = datetime.datetime.today()
    if 'today' in text:
        from_date = today
    elif 'tomorrow' in text:
        from_date = today + datetime.timedelta(days=1)
    elif 'next week' in text:
        from_date = today + datetime.timedelta(days=7)
    else:
        if len(dates)>1:
            try: 
                from_date = dparser.parse(dates[0][1],fuzzy=True)
                to_date = dparser.parse(dates[1][1], fuzzy=True)
            except:
                
        

In [62]:
string = 'on Friday July 10th 2020 I want to leave on a vacation to toronto'
string2 = 'next week, I want to leave on a vacation to toronto tomorrow'

In [63]:
get_from_dates(string2)

[('DATE', 'next week'), ('GPE', 'toronto'), ('DATE', 'tomorrow')]
[('DATE', 'next week'), ('DATE', 'tomorrow')]


In [66]:
import datetime
datetime.datetime.today()

datetime.datetime(2020, 7, 8, 15, 23, 43, 387061)

## Get the dates

In [11]:
# We'll note here that I'll have to consider what happens when the bot asks for the second date. Does it run through this again, or do I create a second function. 
# This will have to be changed depending on the global variables I choose to inlcude

def get_dates(message):
    '''
    This function checks if there are any dates in the message, if so it pulls them out. If there are two dates, then we need to
    parse the list more closely to pull out each date. This function assumes that the date(s) was/were written in a readable form.
    '''
    # Remove punctuation
    clean_message = re.sub(r'[^\w\s]','',message)
    # The first try checks if there are some dates. If there are none or two, it throws a ValueError
    # The second try assumes that there were actually two dates, if this is not true, then it returns nothing.
    try:
        depart_date = dparser.parse(clean_message,fuzzy=True)
        return_date = None
        data_flag = True
    except ValueError:
        try:
            # I will also assume here that the second date given is the return date
            # As a final assumption, we assume that there are only two dates given maximum.
            # Let's first recognize the positions of the dates in the string.
            matcher = Matcher(nlp.vocab)
            matcher.add("DateFinder", None, [{'LOWER':'on'}, {"ENT_TYPE": "DATE"}],
                                          [{'LOWER':'from'}, {"ENT_TYPE": "DATE"}],
                                          [{'LOWER':'to'}, {"ENT_TYPE": "DATE"}],
                                          [{'IS_STOP':True}, {"ENT_TYPE": "DATE"}])
            matches = matcher(nlp(clean_message))
            # First and second date index matchings
            d1 = matches[0]
            d2 = matches[1]
            # Go from first part of date, to one word after (This gets the year, unless there is none, in which case it gets a filler word)
            first_date = nlp(message)[d1[1]:d1[2]+2]
            # If the message ends with the second date, then we need to be careful to not pick up an error
            if (d2[2]==len(clean_message) or d2[2]+1==len(clean_message) or d2[2]+2==len(clean_message)):
                second_date = nlp(clean_message)[d2[1]:]
            else:
                second_date = nlp(clean_message)[d2[1]:d2[2]+2]
            depart_date = dparser.parse(str(first_date),fuzzy=True)
            return_date = dparser.parse(str(second_date),fuzzy=True)
            data_flag=True
        except: 
            return_date = None
            depart_date = None
            data_flag = False

    dates = (depart_date, return_date, data_flag)

    return dates

## Get the locations

In [12]:
def get_locs(message):
  # Remove punctuation
  clean_message = re.sub(r'[^\w\s]','',message)

  from_matcher = Matcher(nlp.vocab)
  from_matcher.add("LocFinder", None, [{'LOWER':'from'},{'ENT_TYPE':'GPE'}],
                                      [{'LOWER':'of'},{'ENT_TYPE':'GPE'}])
  to_matcher = Matcher(nlp.vocab)
  to_matcher.add("LocFinder", None, [{'LOWER':'to'},{'ENT_TYPE':'GPE'}],
                                    [{'LOWER':'visit'},{'ENT_TYPE':'GPE'}],
                                    [{'LOWER':'see'},{'ENT_TYPE':'GPE'}],
                                    [{'LOWER':'out'},{'ENT_TYPE':'GPE'}],
                                    [{'LOWER':'travel'},{'ENT_TYPE':'GPE'}])
  from_match = from_matcher(nlp(clean_message))
  to_match = to_matcher(nlp(clean_message))

  if len(from_match)>0:
    loc1 = from_match[0]
    origin = str(nlp(clean_message)[loc1[1]+1])
  else:
    origin = None
  if len(to_match)>0:
    loc2 = to_match[0]
    dest = str(nlp(clean_message)[loc2[1]+1])
  else:
    dest = None
  
  if (origin == None and dest == None):
    data_flag = False
  else:
    data_flag = True
  locations = (str(origin), str(dest), data_flag)

  return locations

## Get the budget!

In [13]:
def get_budget(message):
  budget_matcher = Matcher(nlp.vocab)
  budget_matcher.add('BudgetFinder', None, [{'LOWER':'of'},{'IS_DIGIT':True}],
                                          [{'LOWER':'spend'},{'IS_DIGIT':True}],
                                          [{'LOWER':'under'},{'IS_DIGIT':True}],
                                          [{'LOWER':'than'},{'IS_DIGIT':True}],
                                          [{'ENT_TYPE':'MONEY'}],
                                          [{'ORTH':'$'},{'IS_DIGIT':True}])
  match = budget_matcher(nlp(message))
  try:
    budget = nlp(message)[match[0][2]-1]
  except IndexError:
    budget = None

  if budget == None:
    data_flag=False
  else:
    data_flag = True
  return (str(budget), data_flag)

##  IATA Codes

In [16]:
city_code = pd.read_csv(f'Airports.csv').groupby('Location').max()
code_dict = city_code.to_dict()['Code']
def iata_code_lookup(city, codes, origin_city=True):
  '''
  This function takes in a city name as well as a dictionary with city names and their IATA codes. It then checks whether this is the origin city or not.
  If the city is in the dictionary, it will output the code. Otherwise, it will output an empty string and an error message that depends on whether or not the city was the origin or destination.
  '''
  try: 
    iata_code = codes[city]
  except KeyError:
    iata_code = ''
    if origin_city:
      print(f'Unfortunately there are no flights leaving {city} at this time.')
      return None
    else:
      print(f'Unfortunately there are no flights to {city} at this time.')
      return None
  return iata_code

## The API stuff

In [30]:
def get_flight(flight_info, codes):
  amadeus = Client(
      client_id=os.getenv("AMADEUS_API_KEY"),
      client_secret=os.getenv("AMADEUS_SECRET")
  )

  try:
      response = amadeus.shopping.flight_offers_search.get(
          originLocationCode = iata_code_lookup(flight_info['origin_loc'], codes, origin_city=True),
          destinationLocationCode = iata_code_lookup(flight_info['dest_loc'], codes, origin_city=False),
          departureDate=f"{flight_info['depart_date'].year}-{str(flight_info['depart_date'].month) if int(flight_info['depart_date'].month)>9 else '0'+str(flight_info['depart_date'].month)}-{str(flight_info['depart_date'].day) if int(flight_info['depart_date'].day)>9 else '0'+str(flight_info['depart_date'].day)}",
          returnDate=f"{flight_info['return_date'].year}-{str(flight_info['return_date'].month) if int(flight_info['return_date'].month)>9 else '0'+str(flight_info['return_date'].month)}-{str(flight_info['return_date'].day) if int(flight_info['return_date'].day)>9 else '0'+str(flight_info['return_date'].day)}",
          maxPrice=int(flight_info['budget']),
          currencyCode='CAD',
          adults=1)
      print(f"Your flight has been booked! Here are the details, tickets will be sent to you by email.")
      print(f"Departure Time: {response.data[0]['itineraries'][0]['segments'][0]['departure']['at']}") 
      print(f"Arrival Time: {response.data[0]['itineraries'][0]['segments'][0]['arrival']['at']}")
      print(f"Total Price: {response.data[0]['price']['total']}")
      print(f"Flight Class: {response.data[0]['travelerPricings'][0]['fareDetailsBySegment'][0]['cabin']}")
  except ResponseError as error:
      print(error)
  return None

## Run the bot

In [31]:
# Now that the chatterbot is trained reasonably, let's create the structure that loops through the conversation, until it has enough information to book a flight.
def Traveler():
  '''
  Due to time constraints, I'm going to let the chatbot do its chatbot thing until it recognizes that a flight is being booked. At that point I'll kick in the script and argue that this is what a 
  human salesman would do...
  '''
  # this list of text queues the chatbot to quit
  byes = ['Goodbye', 'Good bye','You as well, goodbye','Later', 'Bye for now','Bye','See ya','Have a good night','Farewell','Goodbye!', 'bye']
  # First, we'll instantiate a dictionary for the values we want to populate.
  flight_info = {'depart_date':0,
                 'return_date':0,
                 'origin_loc':0,
                 'dest_loc':0,
                 'budget':0
                 }
  conversation_ongoing = True

  while conversation_ongoing:
    user_input = input()
    # If user says something that queues a goodbye message from the chatbot, it gets the chance to say that, and then the loop ends.
    if any(bye in user_input for bye in byes):
      print(random.choice(byes))
      conversation_ongoing = False
      break
    # If there is no information in the travel dictionary, then the chatbot should hold the conversation
    if not ((get_locs(user_input)[2]) or (get_dates(user_input)[2]) or (get_budget(user_input)[1])):
      # Get chatbot response
      print(chatbot.get_response(user_input))
      
    else:
      # Get Locations, if any
      if ((flight_info['dest_loc']==0) and (get_locs(user_input)==None)):
        response_list_dest_loc = ['Where did you want to go?', 'Where were you thinking of travelling to?', 'Where to?']
        print(random.choice(response_list_dest_loc))
        user_input = input()
        locations = get_locs(user_input)
        flight_info['dest_loc']=locations[1]
      else:
        flight_info['dest_loc']=get_locs(user_input)[1]

      if flight_info['origin_loc']==0:
        response_list_origin_loc = ['Where are you leaving from?', 'What city are you flying out of?', 'Where ya coming from?']
        print(random.choice(response_list_origin_loc))
        user_input = input()
        locations = get_locs(user_input)
        flight_info['origin_loc']=locations[0]

      # Get Dates, if any
      if flight_info['depart_date']==0:
        response_list_depart_date = ['When did you want to leave?', 'What dates were you thinking?', 'When is your travel date?', 'For what days should I book the flight?']
        print(random.choice(response_list_depart_date))
        user_input=input()
        dates = get_dates(user_input)
        flight_info['depart_date']=dates[0]

      if flight_info['return_date']==0:
        response_list_return_date = ['When did you want to come back?', 'When do you need to return?', 'What should I set for return date?']
        print(random.choice(response_list_return_date))
        user_input=input()
        dates = get_dates(user_input)
        flight_info['return_date']=dates[0]

      # Get budget if any
      if flight_info['budget']==0:
        response_list_budget = ['How much are you willing to spend on this trip?', "What's your budget for this trip?", "Do you have a price limit for the trip?"]
        print(random.choice(response_list_budget))
        user_input = input()
        budget = get_budget(user_input)
        flight_info['budget'] = budget[0]
      
      if all(flight_info[key] != None for key in flight_info.keys()):
        print(f"So to recap, you want to book a trip from {flight_info['origin_loc']} to {flight_info['dest_loc']} for {flight_info['depart_date']} to {flight_info['return_date']} and you have a budget of {flight_info['budget']}. If this is correct, please say yes, if not, say no")
        user_input = input()
        if (user_input.lower() in ['y', 'yes','yeah', 'yep']):
          print("Excellent, let's, see what the system says. Loading...")
          get_flight(flight_info, code_dict)
        else:
          print("Oh, well thats too bad. Better luck next time!")
          break

      # get number of people, maybe later

  return None

## Demo

In [32]:
Traveller()

hey
Oh hey!
hows it going
Good
how are you
I am on the Internet.
how am i
Radical, let's get to booking! When did you want to leave?
what?
Blue
boo
two boll weevils grew up in s. carolina. one took off to hollywood and became a rich star. the other stayed in carolina and never amounted to much -- and naturally became known as the lesser of two weevils.
I'd like to book a vacation!
I can help you with that. Where are you traveling to?
I'll go to Vancouver
Where are you leaving from?
I'll leave from Toronto
When did you want to leave?
I want to leave tomorrow! so let's say July 8th
What should I set for return date?
Let's set that for August 8th
How much are you willing to spend on this trip?
lets keep it under 3000
So to recap, you want to book a trip from Toronto to Vancouver for 2020-07-08 00:00:00 to 2020-08-08 00:00:00 and you have a budget of 3000. If this is correct, please say yes, if not, say no
yes
Excellent, let's, see what the system says. Loading...
Unfortunately there are n